In [1]:
import inspect
import re
import sys
from io import StringIO

from devtools import debug as _debug

In [2]:
class CustomDebug:
    """
    A wrapper class for the devtools debug function that provides custom output formatting.

    This class wraps the original debug function from the devtools library,
    modifying its output to remove unwanted information (such as ipykernel references)
    and preserve the original debug expressions, including cases with multiple arguments.

    Attributes:
        original_debug (function): The original debug function from devtools.

    Methods:
        __call__(*args, **kwargs): Invokes the debug function with custom output formatting.
        __getattr__(name): Provides access to attributes of the original debug object.

    Usage:
        debug = CustomDebug(_debug)
        debug(expression1, expression2, ...)  # Use like the original debug function
        with debug.timer('label'):  # Use timer and other methods as before
            ...

    The custom output removes ipykernel references and line numbers,
    and replaces argument placeholders with the actual debug expressions used in the code.
    This results in cleaner, more readable debug output, especially useful
    in Jupyter notebook environments.
    """

    def __init__(self, original_debug):
        self.original_debug = original_debug

    def __call__(self, *args, **kwargs):
        # Get the calling frame
        frame = inspect.currentframe().f_back
        # Get the line of code that called debug
        context = inspect.getframeinfo(frame).code_context
        if context:
            # Extract the arguments passed to debug
            call_line = context[0].strip()
            debug_args = call_line[call_line.index('debug(') + 6:].rstrip(')').strip()
        else:
            debug_args = 'unknown'

        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        self.original_debug(*args, **kwargs)
        
        output = sys.stdout.getvalue()
        sys.stdout = old_stdout

        # Clean the output
        lines = output.split('\n')
        cleaned_lines = []
        arg_index = 0
        for line in lines:
            if 'ipykernel' not in line:
                # Remove line numbers and module info
                line = re.sub(r'^.*?<module>\s*', '', line)
                # Replace argument placeholders with actual expressions
                if line.strip().startswith('args['):
                    arg_expressions = [arg.strip() for arg in debug_args.split(',')]
                    if arg_index < len(arg_expressions):
                        line = line.replace(f'args[{arg_index}]', arg_expressions[arg_index])
                        arg_index += 1
                cleaned_lines.append(line)

        cleaned_output = '\n'.join(cleaned_lines).strip()
        print(cleaned_output)

    def __getattr__(self, name):
        return getattr(self.original_debug, name)

# Replace the original debug function
debug = CustomDebug(_debug)

## Example 1


Key differences and advantages of `debug()`:

- Formatting: `debug()` provides a well-indented, multi-line output that's much easier to read than the single-line output from print().
- Custom Classes: Both `print()` and `debug()` respect the custom `__repr__` of the `Employee` class, but `debug()` presents them in a more readable format.
- Complex Types: The `datetime.date` object and `numpy.array` are displayed clearly in the `debug()` output, making it easy to understand their types and values.
- Nested Structures: The nested dictionary (`revenue`) and list (`products`) are clearly formatted in the `debug()` output, improving readability.
- Context Information: debug() includes the file name and line number, which is extremely helpful when debugging larger code-bases.
- Colour Highlighting: In a terminal, `debug()` would also provide syntax highlighting, further enhancing readability (not shown in this text-based example).

In [7]:
import datetime
import numpy as np

In [8]:
class Employee:
    def __init__(self, name, role):
        self.name = name
        self.role = role

    def __repr__(self):
        return f"Employee('{self.name}', '{self.role}')"


company_data = {
    "name": "TechInnovate Inc.",
    "founded": datetime.date(2010, 5, 15),
    "employees": [
        Employee("Alice Johnson", "Software Engineer"),
        Employee("Bob Smith", "Data Scientist"),
    ],
    "revenue": {"2022": 1500000, "2023": 2000000},
    "products": ["AI Assistant", "Smart Analytics", "Cloud Services"],
    "market_share": np.array([0.05, 0.07, 0.06, 0.08]),
    "active": True,
}

In [13]:
print("Using print():\n")
print(company_data)

Using print():

{'name': 'TechInnovate Inc.', 'founded': datetime.date(2010, 5, 15), 'employees': [Employee('Alice Johnson', 'Software Engineer'), Employee('Bob Smith', 'Data Scientist')], 'revenue': {'2022': 1500000, '2023': 2000000}, 'products': ['AI Assistant', 'Smart Analytics', 'Cloud Services'], 'market_share': array([0.05, 0.07, 0.06, 0.08]), 'active': True}


In [12]:
print("\nUsing debug():\n")
debug(company_data)


Using debug():

company_data: {
        'name': 'TechInnovate Inc.',
        'founded': datetime.date(2010, 5, 15),
        'employees': [
            Employee('Alice Johnson', 'Software Engineer'),
            Employee('Bob Smith', 'Data Scientist'),
        ],
        'revenue': {
            '2022': 1500000,
            '2023': 2000000,
        },
        'products': [
            'AI Assistant',
            'Smart Analytics',
            'Cloud Services',
        ],
        'market_share': array([0.05, 0.07, 0.06, 0.08]),
        'active': True,
    } (dict) len=7


## Example 2

In [3]:
import time

In [4]:
x = [1, 2, 3]
debug(x)

x: [1, 2, 3] (list) len=3


In [5]:
def process_data(data):
    time.sleep(2.1)  # Simulating some processing time
    return [x * 2 for x in data]

In [6]:
large_dataset = list(range(1000))

with debug.timer("Data Processing"):
    result = process_data(large_dataset)

debug(len(result))

Data Processing: 2.103s elapsed
len(result: 1000 (int)


## Example 3:

Complex number example


In [14]:
class ComplexNumber:
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag

    def __repr__(self):
        return f"ComplexNumber({self.real}, {self.imag})"


numbers = [ComplexNumber(1, 2), ComplexNumber(3, -4)]
debug(numbers)

numbers: [
        ComplexNumber(1, 2),
        ComplexNumber(3, -4),
    ] (list) len=2


## Examples from Documentation

Eg. A.

In [15]:
v1 = {
    'foo': {1: 'nested', 2: 'dict'},
    'bar': ['apple', 'banana', 'carrot', 'grapefruit'],
}

debug(v1, sum(range(5)))

v1, sum(range(5: {
        'foo': {
            1: 'nested',
            2: 'dict',
        },
        'bar': [
            'apple',
            'banana',
            'carrot',
            'grapefruit',
        ],
    } (dict) len=2


In [ ]:
docs/examples/example.py:8 <module>
    v1: {
        'foo': {
            1: 'nested',
            2: 'dict',
        },
        'bar': [
            'apple',
            'banana',
            'carrot',
            'grapefruit',
        ],
    } (dict) len=2
    sum(range(5)): 10 (int)